# Transfer Learning with Ensemble Method 
The first part leverages four pretrained models for transfer learning, including VGG16, Xception, InceptionV16, and MobileNetV2. After training several epochs to achieve a certain accuracy rate, an ensemble method is used to average out the predictions.

## 1. Import relevent packages 

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
import shutil
from PIL import Image
import tensorflow as tf
import tensorflow.keras.backend as K
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model, load_model
from keras.applications import InceptionV3,VGG16, MobileNetV2, Xception
from keras.layers import *
from keras.optimizers import *
from keras.regularizers import *
from keras.callbacks import *

## 2. Define functions to process data and complie models
As the project was accelerated with GPU, the directory was set on Kaggle notebook. 

In [ ]:
def process_data():
    """
        Assign images to respective foldfers based on their class.
    """
    
    os.mkdir("/kaggle/working/train_set/")
    train_labels = pd.read_csv("../input/final-project-food-recognition-challenge/train_labels.csv")
    
    for folder in train_labels["label"].unique():
        os.mkdir("/kaggle/working/train_set/"+str(folder))

    for index, row in train_labels.iterrows():
        shutil.copy(src="../input/final-project-food-recognition-challenge/train_set/train_set/"+row["img_name"], 
                    dst="/kaggle/working/train_set/"+str(row["label"])+"/")

        
def set_generator():
    """
        Set up image generator for data augmentation.  
    """

    train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        rotation_range=10,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.2,
        zoom_range=0.2,
        channel_shift_range=10,
        horizontal_flip=True,
        fill_mode='constant',
        validation_split=0.2)

    
    directory = "/kaggle/working/train_set/"
    img_height, img_width = 255, 255
    batch_size = 30

    train_generator = train_datagen.flow_from_directory(
        directory,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        shuffle = True, 
        class_mode='categorical',
        subset='training')

    validation_generator = train_datagen.flow_from_directory(
        directory,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        shuffle = True,
        class_mode='categorical',
        subset='validation')
    
    nb_train_samples = train_generator.n
    nb_validation_samples = validation_generator.n
    n_classes = train_generator.num_classes
    
    return train_generator, validation_generator, nb_train_samples, nb_validation_samples, n_classes


def transfer_model(base_model, model_name):
    """
        Freeze the top layers and add some extra layers to train parameters,
        and finally compile the model for transfer learning. 
    """
    
    for layer in base_model.layers[:10]:
        layer.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(80, activation='softmax')(x)
    model = Model(inputs=base_model.inputs, outputs=predictions)

    model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    
    return model


def train_model(model, model_name, epoch_num):
    """
        Train the pre-trained model according to the epoches needed.
    """
    
    batch_size = 30
    checkpointer = ModelCheckpoint(filepath='food_classification_'+ model_name +'.hdf5',
                                   verbose=1,
                                   save_best_only=True)
    
    history = model.fit_generator(train_generator,
                               steps_per_epoch = nb_train_samples // batch_size,
                               validation_data = validation_generator,
                               validation_steps = nb_validation_samples // batch_size,
                               epochs = epoch_num,
                               verbose = 1,
                               callbacks = [checkpointer, es]
                              )
    
    outputdir = "/kaggle/working/"
    model.save(os.path.join(outputdir, f'{model_name}_{epoch_num}.hdf5'))
    
    return history 


def predict_labels(model):

    test_datagen = ImageDataGenerator(rescale=1. / 255)
    batch_size= 30

    pred_dir='../input/final-project-food-recognition-challenge/test_set'

    test_generator = test_datagen.flow_from_directory(
        directory=pred_dir,
        color_mode="rgb",
        batch_size= batch_size,
        class_mode=None,
        shuffle=False
    )

    test_generator.reset()
    
    pred=model.predict_generator(test_generator,verbose=1, 
                             steps=7653/batch_size)

    predicted_class_indices=np.argmax(pred,axis=1)

    labels = (train_generator.class_indices)
    labels = dict((v,k) for k,v in labels.items())
    predictions = [labels[k] for k in predicted_class_indices]

    filenames=test_generator.filenames
    results=pd.DataFrame({"Filename":filenames,
                          "Predictions":predictions})

    results.Filename = results.Filename.apply(lambda x: x.replace("test_set/test_", "").replace(".jpg", "")).astype("int")
    results = results.sort_values(by='Filename')
    
    return results

def plot_accuracy_loss(history):

    fig = plt.figure(figsize=(10,5))

    # Plot accuracy
    plt.subplot(221)
    plt.plot(history.history['accuracy'],'bo--', label = "acc")
    plt.plot(history.history['val_accuracy'], 'ro--', label = "val_acc")
    plt.title("train_acc vs val_acc")
    plt.ylabel("accuracy")
    plt.xlabel("epochs")
    plt.legend()

    # Plot loss function
    plt.subplot(222)
    plt.plot(history.history['loss'],'bo--', label = "loss")
    plt.plot(history.history['val_loss'], 'ro--', label = "val_loss")
    plt.title("train_loss vs val_loss")
    plt.ylabel("loss")
    plt.xlabel("epochs")

    plt.legend()
    plt.show()

## 3. Load the pretrained models for transfer learning and predict

In [ ]:
process_data()
train_generator, validation_generator, nb_train_samples, nb_validation_samples, n_classes = set_generator()

img_height, img_width = 255, 255
batch_size = 30

base_model_1 = Xception(weights='imagenet', include_top=False),
model_name_1 = "Xception"
model_1 = transfer_model(base_model_1, model_name_1)
hist_1 = train_model(model_1, model_name_1, 50)

base_model_2 = MobileNetV2(weights='imagenet', include_top=False)
model_name_2 = "MobileNetV2"
model_2 = transfer_model(base_model_2, model_name_1)
hist_2 = train_model(model_2, model_name_2, 70)

base_model_3 = VGG16(weights='imagenet', include_top=False)
model_name_3 = 'VGG16'
model_3 = transfer_model(base_model_3, model_name_3)
hist_3 = train_model(model_3, model_name_3, 50)

base_model_4 = InceptionV3(weights='imagenet', include_top=False)
model_name_4 = 'InceptionV3'
model_4 = transfer_model(base_model_4, model_name_4)
hist_4 = train_model(model_4, model_name_4, 50)

## 4. Assemble all the transfer learning models for final predictions

In [ ]:
model_1_path = '../input/inception-best/best_model_food101_InceptionV3_1209.hdf5'
model_1 = load_model(model_1_path)
model_1 = Model(inputs=model_1.inputs,
                outputs=model_1.outputs,
                name= model_name_1)

model_2_path = '../input/xception-30/Xception_30.h5'
model_2 = load_model(model_2_path)
model_2 = Model(inputs=model_2.inputs,
                outputs=model_2.outputs,
                name= model_name_2)

model_3_path = '../input/mobilenet-100/MobileNet_100.h5'
model_3 = load_model(model_3_path)
model_3 = Model(inputs=model_3.inputs,
                outputs=model_3.outputs,
                name=model_name_3)

model_4_path = '../input/vgg-45/VGG_45.h5'
model_4 = load_model(model_4_path)
model_4 = Model(inputs=model_4.inputs,
                outputs=model_4.outputs,
                name=model_name_4)

models = [model_1, model_2, model_3, model_4]

model_input = Input(shape=(img_width, img_height, 3))
model_outputs = [model(model_input) for model in models]

ensemble_output = Average()(model_outputs) 
ensemble_model = Model(inputs=model_input, outputs=ensemble_output, name='ensemble')

final_pred = predict_labels(ensemble_model)
final_pred.to_csv('submission.csv')